In [1]:
import ib_insync
from ib_insync import *

import pickle
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import numpy as np
import os

In [2]:
TICKERS = ["SPY", "SH", "SDS", "UPRO", "SSO", "SPXU"]
LEVERAGES = {"SPY": "1x", "SSO": "2x", "UPRO": "3x", "SH":"-1x", "SDS":"-2x", "SPXU": "-3x"}

In [3]:
print(ib_insync.__all__)
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=1)

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescription', 'Sca

<IB connected to 127.0.0.1:4002 clientId=1>

In [4]:
contract1 = Stock('SPXU', 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)
t1 = ib.ticker(contract1)

Error 354, reqId 3: Requested market data is not subscribed.Delayed market data is available.Error&BEST/STK/Top&BEST/STK/Top, contract: Stock(symbol='SPXU', exchange='SMART', currency='USD')


In [44]:
print(t1.bid, t1.bidSize, t1.ask, t1.askSize)

18.65 361 18.66 201


In [48]:
print(t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None))

2021-06-24 21:47:39.384887+08:00


In [55]:
contract2 = Stock('SDS', 'SMART', 'USD')
ib.reqMktData(contract2, '', False, False)
t2 = ib.ticker(contract2)

In [59]:
print(t2.bid, t2.bidSize, t2.ask, t2.askSize)

9.19 700 9.2 4828


In [66]:
del t1
del t2

In [4]:
ib.reqMarketDataType(3)

In [5]:
contract1 = Stock(TICKERS[0], 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)

contract2 = Stock(TICKERS[1], 'SMART', 'USD')
ib.reqMktData(contract2, '', False, False)

contract3 = Stock(TICKERS[2], 'SMART', 'USD')
ib.reqMktData(contract3, '', False, False)

contract4 = Stock(TICKERS[3], 'SMART', 'USD')
ib.reqMktData(contract4, '', False, False)

contract5 = Stock(TICKERS[4], 'SMART', 'USD')
ib.reqMktData(contract5, '', False, False)

contract6 = Stock(TICKERS[5], 'SMART', 'USD')
ib.reqMktData(contract6, '', False, False)

t1 = ib.ticker(contract1)
t2 = ib.ticker(contract2)
t3 = ib.ticker(contract3)
t4 = ib.ticker(contract4)
t5 = ib.ticker(contract5)
t6 = ib.ticker(contract6)

In [6]:
print(t1.bid)

433.57


In [7]:
def read_multiple_prices(tickers, minutes):
#     contracts = list()
#     ib_tickers = list()
    
#     for ticker in tickers:
#         contract = Stock(ticker, 'SMART', 'USD')
# #         contracts.append(contract)
#         ib.reqMktData(contract, '', False, False)
#         ib_tickers.append(ib.ticker(contract))
# #         print(ib.tickers[0]bid)
    
# #     print(tickers[0])

    ib.sleep(0.06)
#     print(t1)
#     print(t1.bid, t1.bidSize, t1.ask, t1.askSize)
#     print(t2.bid, t2.bidSize, t2.ask, t2.askSize)
    
    data = []
    ten_min_data = []
    pre_row_data = None
    
    t_start = time.time()
    prev_time, next_time = t_start, t_start + 60 * 10
    t_end = t_start + 60 * minutes
    print(datetime.now())
    
    while time.time() < t_end:
        if time.time() > next_time:
            with open("data/{}_{}.pkl".format(prev_time, next_time), 'wb') as f:
                pickle.dump(ten_min_data, f)
            prev_time, next_time = next_time, next_time + 60 * 10
            ten_min_data = []
            
        ib.sleep(0.05)
        
        row_data = []
#         print(t1.bid, t1.bidSize, t1.ask, t1.askSize)
        row_data.extend([t1.bid, t1.bidSize, t1.ask, t1.askSize])
        row_data.extend([t2.bid, t2.bidSize, t2.ask, t2.askSize])
        row_data.extend([t3.bid, t3.bidSize, t3.ask, t3.askSize])
        row_data.extend([t4.bid, t4.bidSize, t4.ask, t4.askSize])
        row_data.extend([t5.bid, t5.bidSize, t5.ask, t5.askSize])
        row_data.extend([t6.bid, t6.bidSize, t6.ask, t6.askSize])
        
        if row_data != pre_row_data:
            print(row_data)
            row_time = t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None)
            row = [row_time] + row_data
            data.append(row)
            ten_min_data.append(row)
            pre_row_data = row_data
        else:
            pass
        
    columns=['time']
    for i in range(1, 7):
        columns.extend(['bid{}'.format(i), 'bid_size{}'.format(i), 'ask{}'.format(i), 'ask_size{}'.format(i)])
    
    return pd.DataFrame(data, columns=columns)

In [8]:
%time
df = read_multiple_prices(TICKERS, 600)
df.to_csv('six_asset_data_0706_1.csv')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
2021-07-06 09:25:02.081932
[433.55, 5, 433.58, 39, 15.19, 3339, 15.2, 3315, 8.82, 1761, 8.83, 3055, 117.73, 13, 117.75, 13, 122.33, 4, 122.35, 2, 17.52, 257, 17.53, 105]
[433.55, 5, 433.58, 39, 15.19, 3339, 15.2, 3315, 8.82, 1761, 8.83, 3055, 117.73, 13, 117.75, 13, 122.34, 6, 122.36, 5, 17.52, 237, 17.53, 85]
[433.56, 10, 433.58, 2, 15.19, 3339, 15.2, 3387, 8.82, 1761, 8.83, 3055, 117.74, 3, 117.76, 11, 122.34, 6, 122.36, 5, 17.52, 237, 17.53, 85]
[433.54, 4, 433.56, 3, 15.19, 3509, 15.2, 3028, 8.82, 2231, 8.83, 3055, 117.72, 12, 117.74, 10, 122.33, 3, 122.35, 8, 17.53, 1, 17.53, 60]
[433.54, 4, 433.56, 3, 15.19, 3679, 15.2, 3315, 8.82, 1824, 8.83, 3055, 117.72, 12, 117.74, 10, 122.33, 3, 122.35, 8, 17.53, 1, 17.53, 60]
[433.54, 4, 433.56, 3, 15.19, 3679, 15.2, 3315, 8.82, 1824, 8.83, 3055, 117.72, 12, 117.74, 10, 122.33, 5, 122.35, 5, 17.52, 277, 17.53, 71]
[433.56, 4, 433.58, 6, 15.19, 3679, 15.2, 3315, 8.82, 1824, 8.83

[433.51, 1, 433.52, 5, 15.19, 5453, 15.2, 2998, 8.82, 2328, 8.83, 2893, 117.69, 8, 117.72, 11, 122.3, 4, 122.33, 1, 17.53, 268, 17.54, 270]
[433.51, 1, 433.52, 5, 15.19, 5453, 15.2, 2998, 8.82, 2390, 8.83, 2893, 117.69, 8, 117.72, 11, 122.3, 4, 122.33, 1, 17.53, 268, 17.54, 270]
[433.5, 1, 433.51, 4, 15.19, 5453, 15.2, 2492, 8.82, 2390, 8.83, 2893, 117.68, 13, 117.7, 10, 122.3, 3, 122.33, 8, 17.53, 273, 17.54, 162]
[433.5, 1, 433.51, 4, 15.19, 5453, 15.2, 2492, 8.82, 2390, 8.83, 2893, 117.68, 13, 117.7, 10, 122.3, 3, 122.32, 5, 17.53, 273, 17.54, 162]
[433.5, 1, 433.51, 4, 15.19, 5453, 15.2, 2631, 8.82, 2390, 8.83, 2893, 117.68, 13, 117.7, 10, 122.3, 3, 122.32, 5, 17.53, 273, 17.54, 162]
[433.5, 1, 433.51, 4, 15.19, 5453, 15.2, 2631, 8.82, 2390, 8.83, 2893, 117.68, 13, 117.7, 10, 122.3, 3, 122.32, 5, 17.53, 273, 17.54, 183]
[433.5, 6, 433.51, 99, 15.19, 5453, 15.2, 2631, 8.82, 2390, 8.83, 2893, 117.68, 12, 117.7, 12, 122.3, 3, 122.32, 5, 17.53, 273, 17.54, 183]
[433.5, 6, 433.51, 99, 1

[433.55, 8, 433.58, 14, 15.19, 6376, 15.2, 3479, 8.82, 2019, 8.83, 3098, 117.73, 6, 117.74, 1, 122.33, 5, 122.35, 9, 17.52, 319, 17.53, 126]
[433.55, 9, 433.58, 11, 15.19, 6376, 15.2, 3479, 8.82, 2019, 8.83, 3098, 117.73, 6, 117.74, 1, 122.33, 5, 122.35, 9, 17.52, 319, 17.53, 126]
[433.56, 6, 433.58, 10, 15.19, 6376, 15.2, 3529, 8.82, 2019, 8.83, 3098, 117.73, 6, 117.74, 1, 122.33, 5, 122.35, 9, 17.52, 319, 17.53, 126]
[433.56, 6, 433.58, 10, 15.19, 6376, 15.2, 3529, 8.82, 2019, 8.83, 3098, 117.73, 1, 117.74, 1, 122.33, 5, 122.35, 9, 17.52, 319, 17.53, 126]
[433.56, 6, 433.58, 10, 15.19, 5962, 15.2, 3601, 8.82, 2019, 8.83, 3098, 117.73, 1, 117.74, 1, 122.35, 1, 122.35, 9, 17.52, 304, 17.53, 252]
[433.57, 39, 433.58, 1, 15.19, 5962, 15.2, 3601, 8.82, 2019, 8.83, 3098, 117.74, 12, 117.77, 2, 122.35, 1, 122.35, 9, 17.52, 304, 17.53, 252]
[433.57, 39, 433.58, 1, 15.19, 5962, 15.2, 3601, 8.82, 2019, 8.83, 3098, 117.74, 12, 117.77, 2, 122.34, 1, 122.35, 9, 17.52, 309, 17.53, 186]
[433.57, 39

[433.63, 7, 433.65, 16, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.79, 10, 117.8, 8, 122.38, 4, 122.4, 6, 17.52, 1, 17.53, 350]
[433.63, 7, 433.65, 16, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.79, 13, 117.8, 8, 122.38, 4, 122.4, 6, 17.52, 1, 17.53, 350]
[433.63, 13, 433.65, 8, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.79, 13, 117.8, 8, 122.38, 4, 122.4, 6, 17.52, 1, 17.53, 350]
[433.63, 13, 433.65, 8, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.79, 10, 117.8, 8, 122.38, 4, 122.4, 6, 17.52, 1, 17.53, 350]
[433.64, 2, 433.65, 10, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.79, 10, 117.8, 8, 122.38, 4, 122.4, 6, 17.52, 1, 17.53, 350]
[433.65, 23, 433.67, 41, 15.19, 2861, 15.2, 4175, 8.82, 153, 8.82, 215, 117.8, 20, 117.82, 6, 122.39, 8, 122.41, 6, 17.52, 1, 17.53, 370]
[433.65, 23, 433.67, 41, 15.19, 2861, 15.2, 4515, 8.82, 153, 8.82, 215, 117.8, 20, 117.82, 6, 122.39, 8, 122.41, 6, 17.52, 1, 17.53, 370]
[433.65, 23, 433.67, 41, 15.19, 2861, 15.2, 

[433.72, 38, 433.74, 94, 15.19, 1694, 15.2, 4568, 8.81, 3028, 8.82, 2540, 117.87, 10, 117.88, 4, 122.43, 42, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 22, 433.73, 41, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2540, 117.86, 18, 117.88, 5, 122.43, 37, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 22, 433.73, 41, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2528, 117.86, 18, 117.88, 5, 122.43, 37, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 22, 433.73, 41, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2528, 117.86, 18, 117.88, 5, 122.43, 14, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 22, 433.73, 41, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2528, 117.86, 18, 117.88, 11, 122.43, 14, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 17, 433.73, 32, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2528, 117.86, 18, 117.88, 11, 122.43, 14, 122.45, 6, 17.5, 287, 17.51, 218]
[433.72, 45, 433.73, 32, 15.19, 1740, 15.2, 4568, 8.81, 3028, 8.82, 2528, 117.87, 10, 117.88, 1, 122.43, 14, 122.45, 6, 17.5, 287, 17.51, 21

...

[432.17, 6, 432.2, 8, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.55, 9, 116.66, 8, 121.52, 9, 121.62, 10, 17.68, 41, 17.7, 53]
[432.19, 1, 432.2, 8, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.55, 9, 116.66, 8, 121.52, 9, 121.62, 10, 17.68, 41, 17.7, 53]
[432.19, 1, 432.2, 8, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.55, 9, 116.66, 8, 121.52, 9, 121.62, 10, 17.68, 41, 17.7, 92]
[432.19, 1, 432.2, 8, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.55, 9, 116.66, 8, 121.51, 10, 121.62, 10, 17.68, 41, 17.7, 53]
[432.19, 1, 432.2, 8, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.54, 4, 116.66, 13, 121.51, 10, 121.62, 10, 17.68, 41, 17.7, 53]
[432.17, 1, 432.19, 4, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.54, 4, 116.66, 13, 121.51, 10, 121.62, 10, 17.68, 41, 17.7, 53]
[432.17, 1, 432.18, 3, 15.24, 320, 15.25, 320, 8.87, 281, 8.88, 121, 116.54, 4, 116.65, 13, 121.51, 10, 121.62, 10, 17.69, 10, 17.7, 53]
[432.17, 1, 432.18, 3, 15.24, 320, 15.25, 320, 8.87, 

[432.28, 56, 432.3, 30, 15.24, 1809, 15.25, 5681, 8.87, 1188, 8.88, 1690, 116.68, 9, 116.7, 9, 121.62, 10, 121.66, 14, 17.68, 30, 17.69, 225]
[432.29, 3, 432.3, 22, 15.24, 1809, 15.25, 5681, 8.87, 1188, 8.88, 1690, 116.68, 9, 116.7, 9, 121.62, 10, 121.66, 14, 17.68, 30, 17.69, 225]
[432.28, 42, 432.31, 18, 15.24, 1809, 15.25, 5681, 8.87, 1085, 8.88, 1813, 116.68, 9, 116.72, 9, 121.62, 10, 121.64, 4, 17.68, 40, 17.69, 225]
[432.29, 2, 432.3, 2, 15.24, 1809, 15.25, 5681, 8.87, 1157, 8.88, 1813, 116.68, 9, 116.72, 9, 121.62, 10, 121.64, 4, 17.68, 40, 17.69, 225]
[432.29, 2, 432.3, 5, 15.24, 1809, 15.25, 5681, 8.87, 1157, 8.88, 1813, 116.68, 9, 116.72, 9, 121.62, 10, 121.64, 4, 17.68, 40, 17.69, 225]
[432.29, 2, 432.3, 22, 15.24, 1809, 15.25, 5681, 8.87, 1157, 8.88, 1813, 116.68, 9, 116.72, 9, 121.62, 10, 121.64, 4, 17.68, 40, 17.69, 225]
[432.29, 2, 432.3, 22, 15.24, 1407, 15.25, 5681, 8.87, 812, 8.88, 1813, 116.68, 9, 116.72, 9, 121.62, 10, 121.64, 4, 17.68, 40, 17.69, 225]
[432.29, 2, 4

[432.33, 2, 432.37, 20, 15.23, 5808, 15.24, 1902, 8.87, 757, 8.88, 1475, 116.72, 9, 116.76, 9, 121.65, 10, 121.68, 10, 17.67, 30, 17.68, 161]
[432.3, 20, 432.35, 5, 15.23, 6093, 15.24, 1899, 8.87, 757, 8.88, 1475, 116.72, 9, 116.76, 9, 121.64, 10, 121.67, 10, 17.67, 154, 17.68, 124]
[432.3, 20, 432.35, 5, 15.23, 6378, 15.24, 1899, 8.87, 757, 8.88, 1475, 116.72, 9, 116.76, 9, 121.64, 10, 121.66, 4, 17.67, 154, 17.68, 124]
[432.3, 20, 432.34, 1, 15.23, 6378, 15.24, 1899, 8.87, 757, 8.88, 1475, 116.72, 9, 116.76, 9, 121.64, 10, 121.66, 4, 17.67, 154, 17.68, 124]
[432.3, 5, 432.35, 54, 15.23, 6663, 15.24, 1329, 8.87, 1249, 8.88, 1844, 116.7, 9, 116.76, 9, 121.62, 10, 121.66, 4, 17.67, 174, 17.68, 20]
[432.3, 5, 432.35, 54, 15.23, 6663, 15.24, 1329, 8.87, 1249, 8.88, 1844, 116.72, 9, 116.76, 9, 121.64, 10, 121.66, 4, 17.67, 174, 17.68, 104]
[432.3, 5, 432.35, 54, 15.23, 6663, 15.24, 1329, 8.87, 879, 8.88, 1475, 116.72, 9, 116.76, 9, 121.64, 10, 121.66, 4, 17.67, 174, 17.68, 104]
[432.3, 39,

[432.45, 40, 432.47, 4, 15.23, 5808, 15.24, 2861, 8.86, 4250, 8.87, 1467, 116.84, 9, 116.85, 1, 121.72, 10, 121.75, 20, 17.66, 40, 17.67, 256]
[432.45, 40, 432.46, 2, 15.23, 5808, 15.24, 2861, 8.86, 4250, 8.87, 1467, 116.84, 9, 116.85, 1, 121.72, 10, 121.75, 20, 17.66, 40, 17.67, 256]
[432.46, 2, 432.49, 4, 15.23, 2880, 15.24, 2861, 8.86, 4096, 8.87, 1467, 116.83, 9, 116.87, 9, 121.72, 10, 121.75, 10, 17.66, 20, 17.67, 256]


KeyboardInterrupt: 

In [ ]:
df2 = read_multiple_prices(TICKERS, 600)
df2.to_csv('six_asset_data_0706_2.csv')

In [14]:
ib.disconnect()

## Process data

In [54]:
def calculate_mid_imb(df, num_asset):
    for i in range(1, num_asset + 1):
        df["mid{}".format(i)] = (df["bid{}".format(i)] + df["ask{}".format(i)]) / 2
        df["imb{}".format(i)] = df["bid_size{}".format(i)] / (df["bid_size{}".format(i)] + df["ask_size{}".format(i)])
    
    return df

In [69]:
df_0624 = pd.read_csv("six_asset_data_0624_1.csv", index_col=0)
df_0624["time"] = df_0624["time"].apply(pd.to_datetime)
df_0624.sort_values(by=['time'])
df_0624.time -= timedelta(hours=12)
df_0624 = calculate_mid_imb(df_0624, len(TICKERS))

In [70]:
df_0624.to_csv("cleaned_csv/0624", index=False)

In [71]:
df_0623_1 = pd.read_csv("six_asset_data11.csv", index_col=0)
df_0623_1["time"] = df_0623_1["time"].apply(pd.to_datetime)
df_0623_1.sort_values(by=['time'])
df_0623_1.time -= timedelta(hours=12)
df_0623_1 = calculate_mid_imb(df_0623_1, len(TICKERS))

df_0623_2 = pd.read_csv("six_asset_data_22.csv", index_col=0)
df_0623_2["time"] = df_0623_2["time"].apply(pd.to_datetime)
df_0623_2.sort_values(by=['time'])
df_0623_2.time -= timedelta(hours=12)
df_0623_2 = calculate_mid_imb(df_0623_1, len(TICKERS))

In [72]:
df = pd.concat([df_0623_1, df_0623_2])
df.to_csv("cleaned_csv/0623", index=False)

### Aggregate a pd dataframe from all pickle files

In [138]:
all_pkl_files = os.listdir("data")

In [139]:
all_pkl_files = [file for file in all_pkl_files if file.startswith("1625")]

In [140]:
# Aggregate .pkl data
# all_pkl_files = os.listdir("data")
all_data = []

columns=['time']
for i in range(1, 7):
    columns.extend(['bid{}'.format(i), 'bid_size{}'.format(i), 'ask{}'.format(i), 'ask_size{}'.format(i)])

for file in all_pkl_files:
    temp = pickle.load(open( os.path.join("data", file), "rb" ))
    all_data.extend(temp)
    
df_from_pickle = pd.DataFrame(all_data, columns=columns)

In [141]:
# df_pair_1 = df_from_pickle 
df_pair_1 = df_from_pickle.filter(regex='1|2|time')
df_pair_1["mid1"] = 0.5 * (df_pair_1["bid1"] + df_pair_1["ask1"])
df_pair_1["mid2"] = 0.5 * (df_pair_1["bid2"] + df_pair_1["ask2"])

<ipython-input-141-558df35a0132>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pair_1["mid1"] = 0.5 * (df_pair_1["bid1"] + df_pair_1["ask1"])
<ipython-input-141-558df35a0132>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pair_1["mid2"] = 0.5 * (df_pair_1["bid2"] + df_pair_1["ask2"])


In [142]:
# df_pair_1.time += timedelta(hours=5)
df_pair_1 = df_pair_1.sort_values(by=['time'])
df_pair_1['time'] = pd.to_datetime(df_pair_1['time']).dt.tz_localize(None)  # Get rid of timezone so that datetime can be compared

start_time = datetime(2021, 7, 6, 9, 30, 0)
end_time = datetime(2021, 7, 6, 16, 0, 0)

df_pair_1.loc[(df_pair_1["time"] >= start_time) & (df_pair_1["time"] <= end_time)]
df_pair_1.to_csv("cleaned_csv/0706.csv", index=False)

In [152]:
df_pair_1.head()

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2,mid1,mid2
3202,2021-07-06 09:24:59.172586,433.55,5,433.58,39,15.19,3339,15.2,3315,433.565,15.195
3203,2021-07-06 09:24:59.172586,433.55,5,433.58,39,15.19,3339,15.2,3315,433.565,15.195
3204,2021-07-06 09:25:09.168021,433.56,10,433.58,2,15.19,3339,15.2,3387,433.570,15.195
3205,2021-07-06 09:25:18.216665,433.54,4,433.56,3,15.19,3509,15.2,3028,433.550,15.195
3206,2021-07-06 09:25:18.216665,433.54,4,433.56,3,15.19,3679,15.2,3315,433.550,15.195


In [147]:
df = pd.read_csv("cleaned_csv/0624")
start_time = datetime(2021, 6, 25, 9, 30, 0)
end_time = datetime(2021, 6, 25, 16, 0, 0)

df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None)
df = df.loc[(df["time"] >= start_time) & (df["time"] <= end_time)].drop_duplicates()
df.to_csv("cleaned_csv/0625_new.csv", index=False)

In [153]:
df.head()

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2,bid3,...,mid2,imb2,mid3,imb3,mid4,imb4,mid5,imb5,mid6,imb6
693,2021-06-25 09:30:09.402665,425.95,27,425.97,85,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.098361,18.505,0.348760
694,2021-06-25 09:30:10.323049,425.95,27,425.97,85,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.098361,18.505,0.349338
695,2021-06-25 09:30:19.334121,425.96,16,425.98,160,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.411765,18.505,0.349338
696,2021-06-25 09:30:25.273061,425.96,16,425.98,160,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.725,0.017857,118.140,0.411765,18.505,0.355219
697,2021-06-25 09:30:29.199610,425.95,1,425.97,59,15.47,3150,15.48,4154,9.14,...,15.475,0.431271,9.145,0.474874,111.725,0.017857,118.135,0.238095,18.505,0.355219


In [ ]:
def plot(y_ticker, x_ticker, period=None, price_type="Close"):
    """ 
    Usage: To do a linear regression of y=SPY on x=SH, call `plot("SPY", "SH")`
    This function plots two graph about the relationship between two tickers, and do OLS with them.
        plot 1: The scatter plot of two etf prices
        plot 2: The lineplot of two etf prices over time
    Arguments:
        y_ticker: ticker of the etf to be the dependent variable
        x_ticker: ticker of the etf to be the independent variable
        period (day): If given, use the last `period` number of closed price 
                    If not given, use all the data since both tickers are traded
    """
    
    y = data[y_ticker][price_type] if y_ticker in data else collect_historical(y_ticker)
    x = data[x_ticker][price_type] if x_ticker in data else collect_historical(x_ticker)
    
    period = min(len(y), len(x)) if period == None else period
    y = y[-period:]
    x = x[-period:]
    
    data_pair = pd.DataFrame()
    data_pair[x_ticker], data_pair[y_ticker], data_pair["Year"] = x, y, list(map(lambda x: str(x.year), x.index))
    
    plt.figure(figsize=(20,8))
    plt.subplot(1,2,1)
    plt.title("Relationship between {} and {}".format(y_ticker, x_ticker), fontsize=20)
    sns.scatterplot(data=data_pair, x=x_ticker, y=y_ticker, hue="Year")
    
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    result = model.fit()
    constant = result.params[0]
    slope = result.params[1]
    
    plt.text((max(x["Close"]) + min(x["Close"])) / 2, 0.95*max(y), r'${}={}*{} + {}$'.format(
        y_ticker, round(slope, 2), x_ticker, round(constant, 2)), fontsize=20, ha='center')
    
    plt.subplot(1,2,2)
    plt.title("Price Movement of {} and {}".format(y_ticker, x_ticker), fontsize=20)
    sns.lineplot(data=data_pair)
    plt.ylabel("Price")
    plt.show()
    plt.close()
    
    return data_pair, slope, constant

In [ ]:
def calculate_parameters(df):  # calculate in-sample beta, constant
    df['mid1']=(df.bid1+df.ask1)/2
    df['mid2']=(df.bid2+df.ask2)/2
    
    X = sm.add_constant(df.mid2)
    res = sm.OLS(df.mid1, X).fit()
    constant=res.params[0]
    slope=res.params[1]
    
    predicted_Y=constant+slope*df.mid2
    df['residuals']=df.mid1-predicted_Y
    
    return slope, constant


def prep_data(df, slope, constant, res_num, imb1_num, imb2_num):  # symmetrize data and cut states  
    df_flip=df.copy()
    df_flip.columns=['time', 'ask1', 'ask_size1', 'bid1', 'bid_size1', 'ask2', 'ask_size2', 'bid2', 'bid_size2',
                     'mid1','mid2','residuals']
    df_flip[['ask1', 'bid1', 'ask2', 'bid2','residuals']]=-df_flip[['ask1', 'bid1', 'ask2', 'bid2','residuals']]

    change1=df.bid1[len(df)-1]-df_flip.bid1[0]-0.01
    change2=df.bid2[len(df)-1]-df_flip.bid2[0]-0.01

    df_flip[['ask1', 'bid1']]=df_flip[['ask1', 'bid1']]+change1
    df_flip[['ask2', 'bid2']]=df_flip[['ask2', 'bid2']]+change2
    
    df_flip['mid1']=(df_flip.bid1+df_flip.ask1)/2
    df_flip['mid2']=(df_flip.bid2+df_flip.ask2)/2
    predicted_Y_flip=constant+slope*df_flip.mid2

    df_flip.time=pd.to_datetime(df_flip.time)
    df_flip.time+=timedelta(hours=5)
    
    df.set_index("time",inplace=True)
    df.index = pd.to_datetime(df.index,utc=True)
    df['imb1']=df.bid_size1/(df.bid_size1+df.ask_size1)
    df['imb2']=df.bid_size2/(df.bid_size2+df.ask_size2)
    df2=df[['residuals','mid1','mid2','imb1','imb2']]
    df2.index = df.index.shift(-10,freq='S')
    df2.columns=['residual_later','mid1_later','mid2_later','imb1_later','imb2_later']
    df = pd.merge_asof(df, df2, left_index=True, right_index=True, direction='forward')
    df['pnl'] = df.residual_later-df.residuals  # forward pnl
    df['mid1_diff'] = df.mid1_later-df.mid1
    df['mid2_diff'] = df.mid2_later-df.mid2
    df.dropna(inplace=True)
    
    df_flip.set_index("time",inplace=True)
    df_flip.index = pd.to_datetime(df_flip.index,utc=True)
    df_flip['imb1']=df_flip.bid_size1/(df_flip.bid_size1+df_flip.ask_size1)
    df_flip['imb2']=df_flip.bid_size2/(df_flip.bid_size2+df_flip.ask_size2)
    df2_flip=df_flip[['residuals','mid1','mid2','imb1','imb2']]
    df2_flip.index = df_flip.index.shift(-10,freq='S')# - timedelta(minutes=5)  # data from 5 minutes later
    df2_flip.columns=['residual_later','mid1_later','mid2_later','imb1_later','imb2_later']
    df_flip = pd.merge_asof(df_flip, df2_flip, left_index=True, right_index=True, direction='forward')
    df_flip['pnl'] = df_flip.residual_later-df_flip.residuals  # forward pnl
    df_flip['mid1_diff'] = df_flip.mid1_later-df_flip.mid1
    df_flip['mid2_diff'] = df_flip.mid2_later-df_flip.mid2
    df_flip.dropna(inplace=True)
    
    df=pd.concat([df,df_flip])

    df.index = pd.to_datetime(df.index,utc=True)
    
    df['residual_bucket'] = pd.cut(df['residuals'], res_num, labels=False)
    
    df['imb1']=df.bid_size1/(df.bid_size1+df.ask_size1)
    df['imb2']=df.bid_size2/(df.bid_size2+df.ask_size2)
    df['imb1_bucket'] = pd.cut(df.imb1, imb1_num, labels=False)
    df['imb2_bucket'] = pd.cut(df.imb2, imb2_num, labels=False)
    return df

In [ ]:
slope, constant = calculate_parameters(df)
df = prep_data(df, slope, constant, res_num, imb1_num, imb2_num)